# Stress models

In this notebook we dive a bit deeper into which stressmodels are available in Pastas. Pastas uses Stressmodels to translate a hydrological stress to a contribution to a observed head time series. There are different stressmodels for unique use-cases. 
In Pastas we have the following StressModels:

In [ ]:
import pastas as ps

ps.stressmodels.__all__

## Attributes
Each stressmodel has the following attributes:
- stress (pandas Series)
- name (str, must be unique within a Model)
- rfunc (Pastas response function instance)

These attributes are optional but can be parsed:
- settings (dict or str)
- metadata (dict)
- up (bool, indication whether contribution to the head is positive or negative, given that stress is positive)
- gain_scale_factor (float, inital value estimator for the gain based on stress)

And these settings are infered based on the stress / settings:
- tmin (Timestamp) 
- tmax (Timestamp)
- freq (str, frequency)
- parameters (pandas DataFrame)

So lets create a test where we read data and add it to the basic StressModel:

In [ ]:
import pandas as pd

prec = pd.read_csv(
    "../examples/data/rain_nb1.csv", index_col=0, parse_dates=True
).squeeze()
evap = pd.read_csv(
    "../examples/data/evap_nb1.csv", index_col=0, parse_dates=True
).squeeze()

In [ ]:
sm = ps.StressModel(
    stress=prec,
    rfunc=ps.Gamma(),
    name="precipitation",
    settings="prec",  #
    metadata={"station": "location_x"},  # default is None
    up=True,  # default (head goes up if it rains)
    gain_scale_factor=None,  # default, retrieved from stress
)

Parsing the settings based with a string is equivalent to getting them from the rcParams which has some build-in logic for up- and downsampling and filling missing data.

In [ ]:
settings = ps.rcParams["timeseries"]["prec"]
settings

Since evap is a positive series (>0), up has to be false because evaporation makes the head go down.

In [ ]:
sm2 = ps.StressModel(
    stress=evap,
    rfunc=ps.Gamma(),
    name="precipitation",
    settings="evap",  #
    metadata={"station": "location_x"},  # default is None
    up=False,  # default (head goes up if it rains)
    gain_scale_factor=None,  # default, retrieved from stress
)

The RechargeModel combines the precipitation and evaporation series giving them the same response in a linear / nonlinear recharge combination.

In [ ]:
rm = ps.RechargeModel(
    prec=prec,
    evap=evap,
    rfunc=ps.Exponential(),  # default
    name="recharge",  # default
    recharge=ps.rch.Linear(),  # default, other options are ps.rch.FlexModel(), ps.rch.Peterson(), ps.rch.Berendrecht()
    settings=("prec", "evap"),  # default
)